In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

import os
from pandas import DataFrame
import pandas as pd
from notebook_utils import NotebookUtilities
import matplotlib.pyplot as plt
import re
import humanize
from datetime import timedelta

nu = NotebookUtilities(data_folder_path=os.path.abspath('../data'))

In [3]:

# Get all CSVs into one data frame
if nu.pickle_exists('frvrs_logs_df'):
    frvrs_logs_df = nu.load_object('frvrs_logs_df')
    print(frvrs_logs_df.shape)
    # df = frvrs_logs_df.sample(4).dropna(axis='columns', how='all')
    # display(df.T)

(832366, 109)



# Time Spent on Task

Are all mass casualty tasks (wound packing, needle decompression, tourniquet application, triage tag determination) similar in difficulty based on time spent?

In [17]:

if nu.pickle_exists('time_spent_df'):
    time_spent_df = nu.load_object('time_spent_df')
else:
    
    # Define lists of action types
    action_types_list = [
        'PATIENT_ENGAGED', 'PULSE_TAKEN', 'INJURY_TREATED', 'TOOL_APPLIED', 'TAG_APPLIED'
    ]
    
    # Define columns for grouping
    groupby_columns = ['session_uuid', 'time_group', 'patient_id']
    
    # Group by patient ID
    gb = frvrs_logs_df.sort_values(['elapsed_time']).groupby(groupby_columns)
    rows_list = []
    for (session_uuid, time_group, patient_id), df1 in gb:
    
        # Get the logger version
        logger_version = df1.logger_version.unique().item()
        
        # Get the first time the patient is noticed
        first_notice = df1.elapsed_time.min()
        previous_time = first_notice
        
        # Get the actions with the patient
        mask_series = (df1.action_type.isin(action_types_list))
        for (action_type, elapsed_time), df2 in df1[mask_series].sort_values('elapsed_time').groupby(
            ['action_type', 'elapsed_time']
        ):
            print(previous_time, elapsed_time, time_group, patient_id, previous_time > elapsed_time)
            # if (previous_time > elapsed_time):
            #     display(df1[mask_series].sort_values('elapsed_time').head(4).dropna(axis='columns', how='all').T)
            #     display(df2.sample(min(4, df2.shape[0])).dropna(axis='columns', how='all').T)
            #     raise
            row_dict = {}
            row_dict['logger_version'] = logger_version
            
            # Add the groupby columns to the row dictionary
            for cn in groupby_columns: row_dict[cn] = eval(cn)
            
            # Add the time spent for each action
            row_dict['first_notice'] = first_notice
            row_dict['action_type'] = action_type
            row_dict['action_time'] = elapsed_time
            row_dict['time_spent'] = elapsed_time - previous_time
            previous_time = elapsed_time
            
            # Add the row dictionary to the list
            rows_list.append(row_dict)
    
    # Create a data frame from the list of row dictionaries
    time_spent_df = DataFrame(rows_list)
    # nu.store_objects(time_spent_df=time_spent_df)

13962 80832 1 Gary_0 Root False
80832 132769 1 Gary_0 Root False
132769 136416 1 Gary_0 Root False
136416 143741 1 Gary_0 Root False
143741 29105 1 Gary_0 Root True
29105 91553 1 Gary_0 Root False
91553 92889 1 Gary_0 Root False
92889 95228 1 Gary_0 Root False
95228 96536 1 Gary_0 Root False
96536 97372 1 Gary_0 Root False
97372 97428 1 Gary_0 Root False
97428 115892 1 Gary_0 Root False
115892 117758 1 Gary_0 Root False
117758 118732 1 Gary_0 Root False
118732 260214 1 Gary_0 Root False
260214 263498 1 Gary_0 Root False
263498 271854 1 Gary_0 Root False
271854 155186 1 Gary_0 Root True
155186 167438 1 Gary_0 Root False
167438 169110 1 Gary_0 Root False
169110 176797 1 Gary_0 Root False
176797 80834 1 Gary_0 Root True
80834 132769 1 Gary_0 Root False
132769 136417 1 Gary_0 Root False
136417 143745 1 Gary_0 Root False
305827 704250 2 Bob_0 Root False
704250 693191 2 Bob_0 Root True
693191 695113 2 Bob_0 Root False
695113 699598 2 Bob_0 Root False
699598 703078 2 Bob_0 Root False
703078 7

In [9]:

# Display a sample of the data frame, dropping columns with all NaN values and transposing it
display(time_spent_df.sample(min(4, time_spent_df.shape[0])).dropna(axis='columns', how='all').T)

,31693,20620,24154,9582
logger_version,1.3,1.3,1.0,1.3
session_uuid,f33f702d-4b79-43d9-b980-42dc10c2b882,9a6d2f32-10fb-40ea-bacc-dd6a7d2f4a13,b52e71ec-050f-49e1-b495-7c36f5537544,401ce554-b556-4a10-88bb-424ed5c22ea8
time_group,2,3,1,2
patient_id,Mike_5 Root,Gary_1 Root,Mike_7 Root,Gloria_8 Root
first_notice,316466,245498,11991,475136
action_type,PULSE_TAKEN,PULSE_TAKEN,PATIENT_ENGAGED,PATIENT_ENGAGED
action_time,394033,373913,156998,900295
time_spent,2367,19268,-13283,-17818
